In [1]:
# for modifying python scripts on the go
%load_ext autoreload
%autoreload 2

In [2]:
from model.model import MatlabLSTM
from model.load_weights import load_matlab_weights
from model.predict import predict_from_sequence

In [3]:
from data_processing.normalize import normalize_list, denormalize_pred
from data_processing.format import format_for_model

In [4]:
from data_storage.storage_manager import Storage

In [5]:
from datetime import datetime
import pandas as pd

## Initialize storage

In [6]:
storage = Storage()

data_processing.extract - live extraction does not seem possible yet, data will be simulated
data_sources.hist_dataset - Retrieving existing merged dataset
data_sources.hist_dataset - Saving merged dataset


## Initialize Model

In [7]:
model = MatlabLSTM()
model = load_matlab_weights ( model )

Loaded shapes:
W_in (200, 6)
W_rec (200, 50)
b_all (200,)
W_fc (50,)
b_fc ()
Weights loaded into PyTorch model


## Loop Start: get input

In [8]:
timestamp = datetime.now()
timestamp = timestamp.replace(second=0, microsecond=0)
timestamp

datetime.datetime(2025, 11, 29, 20, 52)

In [9]:
storage.process_new_input(timestamp)
pd.DataFrame([storage.head])

data_processing.extract - live extraction does not seem possible yet, data will be simulated
data_sources.hist_dataset - Retrieving existing merged dataset
data_sources.hist_dataset - Saving merged dataset


,irradiance,temperature,true_power,pred_power,pred_time,rmse,state
0,0.0,21.4,193.842753,None,None,None,raw


In [10]:
storage.df

,irradiance,temperature,true_power,pred_power,pred_time,rmse,state
timestamp,,,,,,,
2024-07-05 15:01:00,658.0,28.0,16081.516703,None,None,None,raw
2024-07-05 15:11:00,555.0,27.0,11994.365093,None,None,None,raw
2024-07-05 15:21:00,599.0,27.1,16361.028037,None,None,None,raw
2024-07-05 15:31:00,625.0,27.4,10863.984139,None,None,None,raw
2024-07-05 15:41:00,628.0,27.4,12340.530450,None,None,None,raw
...,...,...,...,...,...,...,...
2024-07-29 20:11:00,0.0,22.6,610.302781,None,None,None,raw
2024-07-29 20:21:00,0.0,22.9,494.793622,None,None,None,raw
2024-07-29 20:31:00,0.0,22.6,391.507337,None,None,None,raw


## Loop Middle: transform input

In [11]:
inputs_raw = storage.get_k_latest(2)
inputs_formatted = format_for_model(inputs_raw)
inputs_normalized = normalize_list(inputs_formatted)

In [12]:
inputs_raw

,irradiance,temperature,true_power,pred_power,pred_time,rmse,state
timestamp,,,,,,,
2024-07-29 20:41:00,0.0,22.5,292.298813,None,None,None,raw
2024-07-29 20:51:00,0.0,21.4,193.842753,None,None,None,raw


In [13]:
inputs_formatted

[[0.0, 22.5, 292.29881272, 20, 2, 7],
 [0.0, 21.4, 193.84275264000001, 20, 2, 7]]

In [14]:
inputs_normalized

[[-0.7559973658640439,
  0.491928002973191,
  -0.6963979719366865,
  0.8695652173913043,
  0.16666666666666666,
  0.5454545454545454],
 [-0.7559973658640439,
  0.3058476909993979,
  -0.7139703781765805,
  0.8695652173913043,
  0.16666666666666666,
  0.5454545454545454]]

## Loop end: prediction and update storage

In [15]:
pred_normalized =  predict_from_sequence(model, inputs_normalized)
pred = denormalize_pred(pred_normalized)

In [16]:
pred

26.495685277674966

In [17]:
pred_normalized

-0.7438384294509888

In [19]:
storage.change_head(timestamp, pred, 0.666)

In [21]:
print(storage.head)
storage.df

{'irradiance': -1, 'temperature': -1, 'true_power': -1, 'pred_power': 26.495685277674966, 'pred_time': 0.666, 'rmse': None, 'state': 'prediction'}


,irradiance,temperature,true_power,pred_power,pred_time,rmse,state
2024-07-05 15:01:00,658.0,28.0,16081.516703,None,None,None,raw
2024-07-05 15:11:00,555.0,27.0,11994.365093,None,None,None,raw
2024-07-05 15:21:00,599.0,27.1,16361.028037,None,None,None,raw
2024-07-05 15:31:00,625.0,27.4,10863.984139,None,None,None,raw
2024-07-05 15:41:00,628.0,27.4,12340.530450,None,None,None,raw
...,...,...,...,...,...,...,...
2024-07-29 20:21:00,0.0,22.9,494.793622,None,None,None,raw
2024-07-29 20:31:00,0.0,22.6,391.507337,None,None,None,raw
2024-07-29 20:41:00,0.0,22.5,292.298813,None,None,None,raw
2024-07-29 20:51:00,0.0,21.4,193.842753,None,None,None,raw


# SECOND LOOP

In [22]:
timestamp = datetime.now()
timestamp = timestamp.replace(second=0, microsecond=0)
timestamp

datetime.datetime(2025, 11, 29, 21, 3)

In [23]:
storage.process_new_input(timestamp)
pd.DataFrame([storage.head])

data_processing.extract - live extraction does not seem possible yet, data will be simulated
data_sources.hist_dataset - Retrieving existing merged dataset
data_sources.hist_dataset - Saving merged dataset


,irradiance,temperature,true_power,pred_power,pred_time,rmse,state
0,0.0,24.1,99.194012,26.495685,0.666,72.698326,full


In [24]:
storage.df

,irradiance,temperature,true_power,pred_power,pred_time,rmse,state
2024-07-05 15:01:00,658.0,28.0,16081.516703,None,None,None,raw
2024-07-05 15:11:00,555.0,27.0,11994.365093,None,None,None,raw
2024-07-05 15:21:00,599.0,27.1,16361.028037,None,None,None,raw
2024-07-05 15:31:00,625.0,27.4,10863.984139,None,None,None,raw
2024-07-05 15:41:00,628.0,27.4,12340.530450,None,None,None,raw
...,...,...,...,...,...,...,...
2024-07-29 20:21:00,0.0,22.9,494.793622,None,None,None,raw
2024-07-29 20:31:00,0.0,22.6,391.507337,None,None,None,raw
2024-07-29 20:41:00,0.0,22.5,292.298813,None,None,None,raw
2024-07-29 20:51:00,0.0,21.4,193.842753,None,None,None,raw


## middle

In [25]:
inputs_raw = storage.get_k_latest(2)
inputs_formatted = format_for_model(inputs_raw)
inputs_normalized = normalize_list(inputs_formatted)
inputs_raw

,irradiance,temperature,true_power,pred_power,pred_time,rmse,state
2024-07-29 20:51:00,0.0,21.4,193.842753,None,None,None,raw
2025-11-29 20:52:00,0.0,21.4,193.842753,None,None,None,raw


In [26]:
inputs_formatted

[[0.0, 21.4, 193.84275264000001, 20, 2, 7],
 [0.0, 21.4, 193.84275264000001, 20, 7, 11]]

## end

In [27]:
pred_normalized =  predict_from_sequence(model, inputs_normalized)
pred = denormalize_pred(pred_normalized)
pred

33.098694801925376

In [28]:
pred_normalized

-0.7426599264144897

In [29]:
storage.change_head(timestamp, pred, 0.666)

C:\Users\Sara\Downloads\pv-framework-test\src\data_storage\storage_manager.py:72: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  self.df = pd.concat([self.df, head_df], axis=0)


In [30]:
print(storage.head)
storage.df

{'irradiance': -1, 'temperature': -1, 'true_power': -1, 'pred_power': 33.098694801925376, 'pred_time': 0.666, 'rmse': None, 'state': 'prediction'}


,irradiance,temperature,true_power,pred_power,pred_time,rmse,state
2024-07-05 15:01:00,658.0,28.0,16081.516703,NaN,NaN,NaN,raw
2024-07-05 15:11:00,555.0,27.0,11994.365093,NaN,NaN,NaN,raw
2024-07-05 15:21:00,599.0,27.1,16361.028037,NaN,NaN,NaN,raw
2024-07-05 15:31:00,625.0,27.4,10863.984139,NaN,NaN,NaN,raw
2024-07-05 15:41:00,628.0,27.4,12340.530450,NaN,NaN,NaN,raw
...,...,...,...,...,...,...,...
2024-07-29 20:31:00,0.0,22.6,391.507337,NaN,NaN,NaN,raw
2024-07-29 20:41:00,0.0,22.5,292.298813,NaN,NaN,NaN,raw
2024-07-29 20:51:00,0.0,21.4,193.842753,NaN,NaN,NaN,raw
2025-11-29 20:52:00,0.0,21.4,193.842753,NaN,NaN,NaN,raw
